<a href="https://colab.research.google.com/github/UrviSoni/secure_and_private_AI/blob/master/evaluationg_privacy_L_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

In [0]:
def get_parallel_db(db, remove_index):
  
  return torch.cat((db[0:remove_index],db[remove_index + 1:]))

In [0]:
def get_parallel_dbs(db):
  
  parallel_dbs = list()
  
  for i in range(len(db)):
    pdb = get_parallel_db(db, i)
    parallel_dbs.append(pdb)
    
  return parallel_dbs

In [0]:
def create_db_and_parallels(num_entries):
  
  db = torch.rand(num_entries) > 0.5
  pdbs = get_parallel_dbs(db)
  
  return db, pdbs

In [0]:
db, pdbs = create_db_and_parallels(20)

In [6]:
db

tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
       dtype=torch.uint8)

In [7]:
pdbs

[tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,

In [0]:
def query(db):
  
  return db.sum()

In [9]:
query(db)

tensor(13)

In [10]:
query(pdbs[3])

tensor(12)

In [0]:
full_db_result = query(db)

In [0]:
max_distance = 0
for pdb in pdbs:
  pdb_result = query(pdb)
  
  db_distance = torch.abs(pdb_result - full_db_result)
  
  if(db_distance > max_distance):
    max_distance = db_distance

In [0]:
def sensitivity(query, n_entries=1000):
  
  db, pdbs = create_db_and_parallels(n_entries)
  full_db_result = query(db)
  
  max_distance = 0
  for pdb in pdbs:
    pdb_result = query(pdb)

    db_distance = torch.abs(pdb_result - full_db_result)

    if(db_distance > max_distance):
      max_distance = db_distance
      
  return max_distance

In [14]:
sensitivity(query)

tensor(1)

In [0]:
def query(db):
  
  return db.float().mean()

In [16]:
sensitivity(query)

tensor(0.0005)

In [0]:
def query(db, threshold = 5):
  
  return (db.sum() > threshold).float()

In [25]:
db, pdbs = create_db_and_parallels(10)
db.sum()

tensor(6)

In [23]:
db, pdbs = create_db_and_parallels(10)
query(db)

tensor(0.)

In [26]:
db.sum() > 5

tensor(1, dtype=torch.uint8)

In [0]:
sens_f = sensitivity(query, n_entries = 5)

In [31]:
for i in range(10):
  
  sens_f = sensitivity(query, n_entries = 10)
  print(sens_f)

0
tensor(1.)
0
tensor(1.)
tensor(1.)
0
tensor(1.)
0
0
0


In [0]:
db, _ = create_db_and_parallels(100)

In [0]:
pdb = get_parallel_db(db, remove_index = 10)

In [34]:
db[10] 

tensor(1, dtype=torch.uint8)

In [38]:
sum(db)

tensor(53, dtype=torch.uint8)

In [35]:
# differencing attack using sum query

sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [37]:
# differencing attack using mean query

(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(0.0047)

In [39]:
# differencing attack threshold

(sum(db).float() > 52) - (sum(pdb).float() > 52)

tensor(1, dtype=torch.uint8)